In [1]:
import pandas as pd
import re

In [2]:
train=pd.read_csv('final_train.csv')

In [3]:
train.shape

(93608, 2)

In [4]:
train =  train.drop_duplicates()

In [5]:
train.dropna()

,message,Language
0,Has Angelika Kratzer video messaged me?,English
1,Maricopa County weather forecast for this week,English
2,When will my next alarm start,English
3,text Matthew and Helen that are you free,English
4,what ingredients do is have left,English
...,...,...
93603,เล่นเพลงร็อก,Thai
93604,เล่นเพลงเอ็มมิเน็มให้ฉันฟัง,Thai
93605,ให้ข้อมูลข่าวอัปเดตกับฉัน,Thai
93606,ทำช็อกโกแลตกานาชได้ยังไง,Thai


In [6]:
train.head()

,message,Language
0,Has Angelika Kratzer video messaged me?,English
1,Maricopa County weather forecast for this week,English
2,When will my next alarm start,English
3,text Matthew and Helen that are you free,English
4,what ingredients do is have left,English


In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 92535 entries, 0 to 93607
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   message   92535 non-null  object
 1   Language  92535 non-null  object
dtypes: object(2)
memory usage: 2.1+ MB


In [8]:
X=train['message']
Y=train['Language']

In [9]:
Y.shape

(92535,)

In [10]:
keyword=[]
for text in X:
  text = re.sub(r'[!@#$(),%^*?:;~`0-9]', ' ', text)
  text = re.sub(r'[[]]', ' ', text)
  text.lower()
  keyword.append(text)

<ipython-input-10-c19006026619>:4: FutureWarning: Possible nested set at position 1
  text = re.sub(r'[[]]', ' ', text)


In [11]:
from sklearn.feature_extraction.text import CountVectorizer
cv= CountVectorizer()
X=cv.fit_transform(keyword)

In [12]:
from sklearn.preprocessing import LabelEncoder

In [13]:
le=LabelEncoder()
Y = le.fit_transform(Y)

In [14]:
Y

array([0, 0, 0, ..., 5, 5, 5])

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, train_size=0.4, random_state=47)

In [16]:
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [17]:
def training(model):
  clf = model().fit(X_train, Y_train)
  train_acc = accuracy_score(Y_train, clf.predict(X_train))
  test_acc = accuracy_score(Y_test, clf.predict(X_test))
  print(train_acc, test_acc)
  return clf

In [18]:
svc = training(LinearSVC)
random_forest = training(RandomForestClassifier)
decision_tree= training(DecisionTreeClassifier)


0.9997298319554763 0.9876623259667513
0.9997568487599287 0.9772698618540733
0.9997568487599287 0.9691468093153942


In [29]:
str_in = ["Hola, soy Paras Shenmare."]
str_in = cv.transform(str_in)
str_in

<1x38972 sparse matrix of type '<class 'numpy.int64'>'
	with 2 stored elements in Compressed Sparse Row format>

In [30]:
re = le.inverse_transform(svc.predict(str_in))
re[0]

'Spanish'

In [20]:
!pip install streamlit -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 39.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 24.8 MB/s eta 0:00:00


In [36]:
%%writefile app.py
import streamlit as st
import re
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

@st.cache_data
def main():
  train = pd.read_csv(r"/content/final_train.csv")
  train = train.drop_duplicates()
  train=train.dropna()
  X = train['message']
  Y = train['Language']
  keyword=[]
  for text in X:
    text = re.sub(r'[!@#$(),%^*?:;~`0-9]', ' ', text)
    text = re.sub(r'[[]]', ' ', text)
    text.lower()
    keyword.append(text)
  cv= CountVectorizer()
  X=cv.fit_transform(keyword)
  le=LabelEncoder()
  Y = le.fit_transform(Y)
  X_train, X_test, Y_train, Y_test = train_test_split(X,Y, train_size=0.4, random_state=47)
  model = LinearSVC().fit(X_train, Y_train)

  return model, cv, le

# streamlit code

st.set_page_config(page_title='Multilingual Language System', layout='centered', initial_sidebar_state='auto')
st.title('Multilingual Language System')
st.spinner('Loading...')

model, cv, le = main()

#Language Sentence input
user_text = st.text_input('Enter the Text -->', label_visibility="visible", disabled=False, max_chars=None, key=None, type='default')

result = []
with st.form("Form", clear_on_submit=True):
  submitted = st.form_submit_button('Submit', disabled=not (user_text))
  if submitted:
    with st.spinner('Predicting...'):
      text = []
      text.append(user_text)
      text=cv.transform(text)
      res = le.inverse_transform(model.predict(text))
      result.append(res[0])

if len(result):
  with st.spinner('Predicting...'):
    st.info(f'Predicted Language is {result[0]}')

Overwriting app.py


In [37]:
!streamlit run app.py & npx localtunnel --port 8501

[##................] / fetchMetadata: sill resolveWithNewModule ms@2.1.2 checki


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.86.82.251:8501

npx: installed 22 in 5.082s
your url is: https://five-mirrors-roll.loca.lt
/content/app.py:20: FutureWarning: Possible nested set at position 1
  text = re.sub(r'[[]]', ' ', text)
  Stopping...
^C
